In [5]:
import os
import glob
import numpy as np
import pandas as pd

# ===== CSV があるフォルダ =====
base_dir = "macro_data/country_data/country_modern_rate_npy"

# energydata_*.csv を全部対象にする
csv_files = glob.glob(os.path.join(base_dir, "energydata_*.csv"))
print("見つかったファイル数:", len(csv_files))
for f in csv_files:
    print("  -", f)

if not csv_files:
    raise SystemExit("CSV が見つかりません。パスを確認してください。")

for fpath in csv_files:
    fname = os.path.basename(fpath)
    country = os.path.splitext(fname)[0].replace("energydata_", "")
    print(f"\n=== {country} ===")
    print("  入力ファイル:", fpath)

    # 1) ヘッダーありで普通に読む（1行目がヘッダー）
    df = pd.read_csv(fpath)

    # 2) 「1列目は年」とみなす（列名が Year でも Unnamed: 0 でも何でもよい）
    year_col = df.columns[0]

    # Year を数値にし、NaN 行を落とす（空行やフッターを削除）
    df[year_col] = pd.to_numeric(df[year_col], errors="coerce")
    df = df.dropna(subset=[year_col])
    df[year_col] = df[year_col].astype(int)

    # 3) 他の列も数値にしておく
    for c in df.columns:
        if c == year_col:
            continue
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # 4) Coal 列を探し、そこから右側を modern energy とみなす
    cols = list(df.columns)
    cols_lower = [str(c).strip().lower() for c in cols]
    try:
        idx_coal = cols_lower.index("coal")
    except ValueError:
        print("  'Coal' 列が見つかりません。スキップします。")
        print("  df.columns =", list(df.columns))
        continue

    modern_cols = cols[idx_coal:]                 # Coal 以右すべて
    modern_cols_no_year = [c for c in modern_cols if c != year_col]

    print("  modern energy 列 =", modern_cols_no_year)

    # 5) modern energy rate (0〜1) を計算
    modern_share = df[modern_cols_no_year].sum(axis=1) / 100.0

    # 6) 最大値で正規化
    max_val = modern_share.max()
    if max_val == 0 or np.isnan(max_val):
        modern_share_norm = modern_share * np.nan
    else:
        modern_share_norm = modern_share / max_val

    years = df[year_col].to_numpy(dtype=int)
    modern_rate = modern_share.to_numpy(dtype=float)
    modern_rate_norm = modern_share_norm.to_numpy(dtype=float)

    # 7) npy を保存（CSV と同じフォルダに）
    out_path = os.path.join(base_dir, f"{country}.npy")
    np.save(out_path, {
        "Year": years,
        "modern_rate": modern_rate,
        "modern_rate_norm": modern_rate_norm,
    })

    print("  保存:", out_path)
    print("  年の範囲:", years.min(), "–", years.max())
    print(f"  modern_rate 最大値: {max_val:.3f}")

print("\n全て終了しました。")


見つかったファイル数: 10
  - macro_data/country_data/country_modern_rate_npy/energydata_Canada.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_France.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Uruguay.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Spain.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Germany.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Italy.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Sweden.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Netherland.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Englandwales.csv
  - macro_data/country_data/country_modern_rate_npy/energydata_Portugal.csv

=== Canada ===
  入力ファイル: macro_data/country_data/country_modern_rate_npy/energydata_Canada.csv
  modern energy 列 = ['Coal', 'Oil', 'Natural Gas', 'Primary electricity']
  保存: macro_data/country_data/country_modern_rate_npy/Canada.np

In [11]:
data=np.load("/work/a06/tsuda/2025_master_Tsuda/sotsuron/energy_transition_analysis/macro_data/macro_data/country_data/country_modern_rate_npy/Canada.npy")
print(data)

ValueError: Object arrays cannot be loaded when allow_pickle=False